In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
df_mediated_norm = pd.read_csv('../../data/mediated_schema/mediated_schema_normalized.csv', 
                               dtype={'id_source_vehicles': 'object'})

/tmp/ipykernel_205047/4111612839.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mediated_norm = pd.read_csv('../../data/mediated_schema/mediated_schema_normalized.csv',


In [5]:
df_mediated_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176992 entries, 0 to 3176991
Data columns (total 19 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id_source_vehicles   object 
 1   id_source_used_cars  object 
 2   location             object 
 3   price                float64
 4   year                 float64
 5   manufacturer         object 
 6   model                object 
 7   cylinders            object 
 8   fuel_type            object 
 9   mileage              float64
 10  transmission         object 
 11  vin                  object 
 12  traction             object 
 13  body_type            object 
 14  main_color           object 
 15  description          object 
 16  latitude             float64
 17  longitude            float64
 18  pubblication_date    object 
dtypes: float64(5), object(14)
memory usage: 460.5+ MB


In [6]:
df_mediated_norm.head()

,id_source_vehicles,id_source_used_cars,location,price,year,manufacturer,model,cylinders,fuel_type,mileage,transmission,vin,traction,body_type,main_color,description,latitude,longitude,pubblication_date
0,7316814884,NaN,auburn,33590.0,2014.0,gmc,sierra 1500 crew cab slt,8 cylinders,gas,57923.0,other,3GTP1VEC4EG551563,NaN,pickup,white,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T12:31:18-0500
1,7316814758,NaN,auburn,22590.0,2010.0,chevrolet,silverado 1500,8 cylinders,gas,71229.0,other,1GCSCSE06AZ123805,NaN,pickup,blue,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T12:31:08-0500
2,7316814989,NaN,auburn,39590.0,2020.0,chevrolet,silverado 1500 crew,8 cylinders,gas,19160.0,other,3GCPWCED5LG130317,NaN,pickup,red,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T12:31:25-0500
3,7316743432,NaN,auburn,30990.0,2017.0,toyota,tundra double cab sr,8 cylinders,gas,41124.0,other,5TFRM5F17HX120972,NaN,pickup,red,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T10:41:31-0500
4,7316356412,NaN,auburn,15000.0,2013.0,ford,f150 xlt,6 cylinders,gas,128000.0,automatic,NaN,rwd,truck,black,2013 f 150 xlt v6 4 door good condition leveli...,32.592,-85.5189,2021-05-03T14:02:03-0500


In [7]:
# Pulizia dei record con valore VIN non corretti, per la generazione della Ground Truth

def is_valid_vin_checksum(vin):
    """
    Algoritmo ufficiale per la validazione della 9a cifra (Check Digit) del VIN.
    """
    if not isinstance(vin, str) or len(vin) != 17:
        return False
    
    # Mappa valori caratteri ISO
    val_map = {
        'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8,
        'J':1, 'K':2, 'L':3, 'M':4, 'N':5, 'P':7, 'R':9, 'S':2,
        'T':3, 'U':4, 'V':5, 'W':6, 'X':7, 'Y':8, 'Z':9,
        '0':0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9
    }
    # Pesi per ogni posizione
    weights = [8, 7, 6, 5, 4, 3, 2, 10, 0, 9, 8, 7, 6, 5, 4, 3, 2]
    
    try:
        total = sum(val_map[vin[i]] * weights[i] for i in range(17) if i != 8)
        remainder = total % 11
        check_digit = 'X' if remainder == 10 else str(remainder)
        return vin[8] == check_digit
    except KeyError:
        return False

def count_differences(row1, row2, colonne_da_confrontare):
    diff = 0
    for col in colonne_da_confrontare:
        v1, v2 = row1[col], row2[col]
        if pd.isna(v1) and pd.isna(v2):
            continue
        if v1 != v2:
            diff += 1
    return diff

def deduplica_vin_per_similarita(
    df,
    vin_col='vin',
    date_col='pubblication_date',
    max_diff=3
):
    colonne_da_confrontare = [
        c for c in df.columns
        if c not in {vin_col, date_col, 'id_source_vehicles', 'id_source_used_cars', 'description'}
    ]

    # Convertiamo temporaneamente in una colonna di supporto per l'ordinamento
    # senza sovrascrivere quella originale (che vogliamo preservare come stringa)
    df['temp_date_sort'] = pd.to_datetime(df[date_col], errors='coerce')

    record_finali = []

    for vin, gruppo in df.groupby(vin_col):
        if len(gruppo) == 1:
            record_finali.append(gruppo.iloc[0])
            continue

        # Ordiniamo usando la colonna di supporto
        gruppo = gruppo.sort_values(by='temp_date_sort', ascending=False)
        usati = set()

        for i, row_i in gruppo.iterrows():
            if i in usati:
                continue
            cluster = [i]
            for j, row_j in gruppo.iterrows():
                if j == i or j in usati:
                    continue
                diff = count_differences(row_i, row_j, colonne_da_confrontare)
                if diff <= max_diff:
                    cluster.append(j)
            
            for idx in cluster:
                usati.add(idx)
            
            # Teniamo il record del cluster (ha la data originale)
            record_finali.append(gruppo.loc[cluster].iloc[0])

    # Creiamo il DataFrame e rimuoviamo la colonna di supporto
    df_res = pd.DataFrame(record_finali).reset_index(drop=True)
    if 'temp_date_sort' in df_res.columns:
        df_res = df_res.drop(columns=['temp_date_sort'])
    
    return df_res


def pulizia_vin_avanzata(df, colonna_vin='vin', colonna_data='pubblication_date'):
    """
    Esegue pulizia alfanumerica, rimozione null, validazione checksum e deduplicazione.
    """
    # 1. RIMOZIONE RECORD CON VIN NULLI (Fondamentale per la Ground Truth)
    df_clean = df.dropna(subset=[colonna_vin]).copy()
    
    # 2. NORMALIZZAZIONE E PULIZIA STRINGA
    # Rimuove tutto ciò che non è lettera o numero e rende maiuscolo
    df_clean[colonna_vin] = df_clean[colonna_vin].astype(str).str.upper().str.replace(r'[^A-Z0-9]', '', regex=True)
    
    # 3. FILTRI FORMATO E CARATTERI VIETATI (I, O, Q)
    # Lunghezza 17 e controllo caratteri legali
    regex_legale = r'^[A-HJ-NPR-Z0-9]{17}$'
    df_clean = df_clean[df_clean[colonna_vin].str.contains(regex_legale, regex=True)]
    
    # 4. RIMOZIONE PLACEHOLDER (Pattern ripetuti)
    regex_placeholder = r'^(.)\1{16}$|12345678|ABCDEFGH'
    df_clean = df_clean[~df_clean[colonna_vin].str.contains(regex_placeholder, regex=True)]
    
    # 5. VALIDAZIONE CHECKSUM
    # Applichiamo la funzione is_valid_vin_checksum
    df_clean['vin_valido'] = df_clean[colonna_vin].apply(is_valid_vin_checksum)
    df_clean = df_clean[df_clean['vin_valido'] == True].drop(columns=['vin_valido'])
    
    # 6. DEDUPLICAZIONE AVANZATA VIN
    df_clean = deduplica_vin_per_similarita(df_clean, vin_col=colonna_vin, date_col=colonna_data, max_diff=3)
    
    return df_clean

In [9]:
df_mediated_norm['vin'].value_counts()

vin
1FMJU1JT1HEA52352    261
3C6JR6DT3KG560649    236
1FTER1EH1LLA36301    231
5TFTX4CN3EX042751    227
1GCHTCE37G1186784    214
                    ... 
1C4RJFBG4HC908852      1
1FMCU9H99LUB55938      1
KM8J3CA47MU300479      1
1HGCV1F4XLA111931      1
5FNYF6H52MB004143      1
Name: count, Length: 2964967, dtype: int64

In [8]:
# --- APPLICAZIONE ---

# Applichiamo la funzione al dataset normalizzato per la pulizia dei record con VIN non validi
print(f"Record pre-pulizia: Dataset={len(df_mediated_norm)}")

df_sanificato_norm = pulizia_vin_avanzata(df_mediated_norm)

print(f"Record post-pulizia: Dataset={len(df_sanificato_norm)}")

Record pre-pulizia: Dataset=3176992


/tmp/ipykernel_205047/2200145395.py:107: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_clean = df_clean[~df_clean[colonna_vin].str.contains(regex_placeholder, regex=True)]
/tmp/ipykernel_205047/2200145395.py:51: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['temp_date_sort'] = pd.to_datetime(df[date_col], errors='coerce')


Record post-pulizia: Dataset=2889790


In [11]:
df_sanificato_norm['vin'].value_counts()

vin
KNDJP3A55H7463251    5
1FTYR10U61PA73092    5
1FTEW1EP5HFA09219    5
1FTEW1EF0FFB34035    4
1FTNE24L89DA76367    4
                    ..
1N4BL4CV7KC153917    1
1N4BL4CV7KC154274    1
1N4BL4CV7KC154596    1
1N4BL4CV7KC155098    1
1N4BL4CV7KC145302    1
Name: count, Length: 2958730, dtype: int64

In [9]:
def genera_ground_truth(df_mediato, ratio_negativi=2.0, random_state=42):
    """
    Genera GT includendo match A-B, A-A e B-B.
    Formato: (id_A, attr_A, id_B, attr_B, label)
    """
    np.random.seed(random_state)
    
    attributi = [
        'location', 'price', 'year', 'manufacturer', 'model', 'cylinders',
        'fuel_type', 'mileage', 'transmission', 'traction',
        'body_type', 'main_color', 'description',
        'latitude', 'longitude', 'pubblication_date'
    ]

    # 1. Creiamo un ID unico "canonico" per ogni riga, dato che un record
    # ha o l'id_source_vehicles o l'id_source_used_cars.
    df = df_mediato.copy()
    df['id_univoco'] = df['id_source_vehicles'].fillna(df['id_source_used_cars']).astype(str)
    
    # Teniamo solo le colonne necessarie per il join (ID, VIN e Attributi)
    df_per_join = df[['id_univoco', 'vin'] + attributi]

    # =====================================================
    # 2. GENERAZIONE POSITIVI (Label 1) - Includendo A-A, B-B, A-B
    # =====================================================
    # Self-join sul VIN
    positivi = pd.merge(
        df_per_join,
        df_per_join,
        on='vin',
        suffixes=('_A', '_B')
    )

    # REGOLE DI UNIVOCITÀ E FILTRO:
    # 1. Rimuoviamo il confronto di un record con se stesso (id_A == id_B)
    # 2. Per evitare coppie duplicate invertite (r1,r2 e r2,r1), 
    #    teniamo solo dove id_A è lessicalmente minore di id_B.
    positivi = positivi[positivi['id_univoco_A'] < positivi['id_univoco_B']].copy()

    positivi['label'] = 1
    positivi = positivi.rename(columns={'id_univoco_A': 'id_A', 'id_univoco_B': 'id_B'})
    
    # Rimuoviamo il VIN (Punto 4.B)
    positivi = positivi.drop(columns=['vin'])
    
    n_positivi = len(positivi)
    print(f"Coppie positive totali (A-B, A-A, B-B): {n_positivi}")

    # =====================================================
    # 3. GENERAZIONE NEGATIVI (Label 0)
    # =====================================================
    n_negativi_target = int(n_positivi * ratio_negativi)
    negativi_list = []
    
    # Campionamento casuale per generare negativi
    # Usiamo un approccio a blocchi per velocità
    while len(negativi_list) < n_negativi_target:
        idx1 = np.random.choice(df.index, size=n_negativi_target)
        idx2 = np.random.choice(df.index, size=n_negativi_target)
        
        tmp_A = df.loc[idx1, ['id_univoco', 'vin'] + attributi].reset_index(drop=True)
        tmp_B = df.loc[idx2, ['id_univoco', 'vin'] + attributi].reset_index(drop=True)
        
        # Filtro: ID diversi AND VIN diversi AND ordine univoco
        mask = (tmp_A['id_univoco'] != tmp_B['id_univoco']) & \
               (tmp_A['vin'] != tmp_B['vin'])
        
        # Per i negativi applichiamo lo stesso ordinamento per coerenza
        # Se id_A > id_B, invertiamo i record per mantenere id_A < id_B
        swap_mask = tmp_A['id_univoco'] > tmp_B['id_univoco']
        
        # Applichiamo lo swap dove necessario
        tmp_A.loc[swap_mask], tmp_B.loc[swap_mask] = tmp_B.loc[swap_mask], tmp_A.loc[swap_mask]
        
        batch_neg = pd.concat([
            tmp_A.add_suffix('_A'), 
            tmp_B.add_suffix('_B')
        ], axis=1)[mask]
        
        negativi_list.append(batch_neg)
        combined_neg = pd.concat(negativi_list).drop_duplicates(subset=['id_univoco_A', 'id_univoco_B'])
        if len(combined_neg) >= n_negativi_target:
            negativi = combined_neg.head(n_negativi_target).copy()
            break

    negativi = negativi.rename(columns={'id_univoco_A': 'id_A', 'id_univoco_B': 'id_B'})
    negativi = negativi.drop(columns=['vin_A', 'vin_B'])
    negativi['label'] = 0
    
    print(f"Coppie negative generate: {len(negativi)}")

    # =====================================================
    # 4. ASSEMBLAGGIO FINALE
    # =====================================================
    ground_truth = pd.concat([positivi, negativi], ignore_index=True)
    ground_truth = ground_truth.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Ordinamento finale colonne
    cols_A = ['id_A'] + [f"{a}_A" for a in attributi]
    cols_B = ['id_B'] + [f"{a}_B" for a in attributi]
    ground_truth = ground_truth[cols_A + cols_B + ['label']]

    return ground_truth

In [10]:
# --- ESECUZIONE ---

ground_truth_df = genera_ground_truth(df_sanificato_norm, ratio_negativi=2.0)

# Stampa info finali
print(f"\nShape finale Ground Truth: {ground_truth_df.shape}")
print(f"Distribuzione Label:\n{ground_truth_df['label'].value_counts()}")

Coppie positive totali (A-B, A-A, B-B): 5921
Coppie negative generate: 11842

Shape finale Ground Truth: (17763, 35)
Distribuzione Label:
label
0    11842
1     5921
Name: count, dtype: int64


In [11]:
assert not ground_truth_df.isna().all(axis=1).any(), "Righe completamente vuote!"
assert ground_truth_df.filter(like='_A').isna().all(axis=1).sum() == 0
assert ground_truth_df.filter(like='_B').isna().all(axis=1).sum() == 0

In [12]:
ground_truth_df[ground_truth_df['label']==0].head()

,id_A,location_A,price_A,year_A,manufacturer_A,model_A,cylinders_A,fuel_type_A,mileage_A,transmission_A,traction_A,body_type_A,main_color_A,description_A,latitude_A,longitude_A,pubblication_date_A,id_B,location_B,price_B,year_B,manufacturer_B,model_B,cylinders_B,fuel_type_B,mileage_B,transmission_B,traction_B,body_type_B,main_color_B,description_B,latitude_B,longitude_B,pubblication_date_B,label
0,S2_2485250,las vegas,24905.0,2020.0,honda,civic,i4,gas,0.0,cvt,fwd,sedan,blue,wheels 17 alloy wheels w silver accents vsa el...,36.1435,-115.2590,2020-07-16,S2_2945424,san jose,18998.0,2018.0,chevrolet,equinox,i4,gas,22179.0,automatic,fwd,suv crossover,gray,carmax curbside is now available at select sto...,37.2750,-121.8680,2020-07-30,0
1,S2_131029,burlington,51187.0,2019.0,mercedesbenz,sprinter,i4,gas,19.0,automatic,4x2,van,black,brand new 2019 sprinter 1500 passenger van rig...,42.4930,-71.1940,2019-12-18,S2_993289,columbus,24010.0,2019.0,ford,transit cargo,v6,gas,17093.0,automatic,rwd,van,white,this ford transit van delivers a regular unlea...,40.0608,-82.9686,2020-07-17,0
2,S2_1388009,chillicothe,12988.0,2018.0,chevrolet,equinox,i4,gas,91445.0,automatic,fwd,suv crossover,silver,additional info active grille shutters door ha...,39.7821,-93.5529,2020-03-27,S2_2883314,glendora,17995.0,2020.0,chevrolet,malibu,i4,gas,1.0,cvt,fwd,sedan,black,summer sales event come by and take advantage ...,34.1176,-117.8250,2020-08-02,0
3,S2_1477438,west palm beach,27991.0,2017.0,lexus,nx 200t,i4,gas,26575.0,automatic,NaN,suv crossover,silver,2017 lexus nx 200t silver clean carfax fwd 6 s...,26.7153,-80.1038,2020-09-01,S2_247616,seekonk,12900.0,2012.0,audi,a4,i4,gas,85830.0,automatic,awd,sedan,blue,have confidence when purchasing your next vehi...,41.8062,-71.3329,2020-08-12,0
7,S2_1869433,vincennes,15995.0,2016.0,hyundai,sonata,i4,gas,42420.0,automatic,fwd,sedan,silver,visit vincennes autoplex online at to see more...,38.6926,-87.4983,2020-08-15,S2_463048,westmoreland,3990.0,2001.0,chevrolet,silverado 2500hd,v8,gas,328095.0,automatic,NaN,pickup truck,brown,upstate auto gallery is a family owned and ope...,43.0975,-75.3994,2020-09-06,0


In [13]:
ground_truth_df[ground_truth_df['label']==1].head()

,id_A,location_A,price_A,year_A,manufacturer_A,model_A,cylinders_A,fuel_type_A,mileage_A,transmission_A,traction_A,body_type_A,main_color_A,description_A,latitude_A,longitude_A,pubblication_date_A,id_B,location_B,price_B,year_B,manufacturer_B,model_B,cylinders_B,fuel_type_B,mileage_B,transmission_B,traction_B,body_type_B,main_color_B,description_B,latitude_B,longitude_B,pubblication_date_B,label
4,7304232772,shreveport,0.0,2015.0,kia,optima 4dr sdn lx,4 cylinders,gas,149315.0,automatic,fwd,sedan,silver,see more pictures details the carfax and video...,33.211887,-92.656760,2021-04-09T12:05:04-0500,7305363719,texarkana,0.0,2015.0,kia,optima lx,4 cylinders,gas,149315.0,automatic,rwd,sedan,silver,see more pictures details the carfax and video...,33.211633,-92.656696,2021-04-11T16:48:56-0500,1
5,7304797439,merced,23995.0,2015.0,chevrolet,silverado ls,NaN,gas,74145.0,automatic,NaN,NaN,NaN,automatic 2wd v6 4 3liter 74 145 miles backup ...,NaN,NaN,2021-04-10T11:22:34-0700,7306382067,modesto,23995.0,2015.0,chevrolet,silverado 1500 ls,6 cylinders,gas,74145.0,automatic,NaN,pickup,red,2015 chevrolet silverado ls v6 automatic air c...,37.596846,-120.938695,2021-04-13T14:15:42-0700,1
6,7314437045,tucson,8995.0,2007.0,toyota,prius,4 cylinders,hybrid,69000.0,automatic,fwd,sedan,white,2007 toyota prius 5dr hb sedan drive it home t...,32.250000,-110.886900,2021-04-29T11:30:15-0700,S2_2406396,tucson,8995.0,2007.0,toyota,prius,i4 hybrid,hybrid,69000.0,automatic,fwd,hatchback,white,thank you for visiting jay auto sales where we...,32.236100,-110.895000,2020-05-31,1
13,7313778382,tampa bay area,29900.0,2018.0,ford,f150,6 cylinders,gas,17480.0,automatic,rwd,truck,white,2018 ford f 150 xl supercab 6 5 ft bed 2wd off...,27.986120,-82.270380,2021-04-28T09:36:57-0400,S2_1452915,seffner,29900.0,2018.0,ford,f150,v6 flex fuel vehicle,flex fuel vehicle,17480.0,automatic,4x2,pickup truck,white,nice clean truck carfax 3 3l v 6 engine superc...,27.985400,-82.268900,2020-08-01,1
15,7302648287,pueblo,13995.0,2014.0,kia,sportage,4 cylinders,gas,90101.0,automatic,4wd,suv,blue,this 2014 kia sportage ex has been sold but ca...,38.279325,-104.607754,2021-04-06T10:30:42-0600,S2_2747284,pueblo,12495.0,2014.0,kia,sportage,i4,gas,90101.0,automatic,awd,suv crossover,blue,carpros where quality drives home great financ...,38.279300,-104.607000,2020-03-13,1


In [15]:
ground_truth_df.to_csv('../../data/ground_truth/ground_truth.csv', index=False)

In [16]:
# Split GT → train vs eval (STRATIFICATO)
GT_train, GT_eval = train_test_split(
    ground_truth_df,
    test_size=0.30,
    random_state=42,
    stratify=ground_truth_df['label']
)

print("Distribuzione GT_train:")
print(GT_train['label'].value_counts(normalize=True))

print("\nDistribuzione GT_eval:")
print(GT_eval['label'].value_counts(normalize=True))

print(f"Split completato:")
print(f"- Train: {len(GT_train)} righe")
print(f"- Eval:   {len(GT_eval)} righe")

Distribuzione GT_train:
label
0    0.66664
1    0.33336
Name: proportion, dtype: float64

Distribuzione GT_eval:
label
0    0.666729
1    0.333271
Name: proportion, dtype: float64
Split completato:
- Train: 12434 righe
- Eval:   5329 righe


In [17]:
# Salvataggio su file CSV
GT_eval.to_csv("../../data/ground_truth/GT_eval.csv", index=False)

In [18]:
# Eseguiamo uno split stratificato per mantenere la proporzione tra match (1) e non-match (0)
# Primo split: 70% Train e 30% Temporaneo (che divideremo in Val e Test)
train_set, temp_set = train_test_split(
    GT_train, 
    test_size=0.30, 
    random_state=42, 
    stratify=GT_train['label']
)

# Secondo split: Dividiamo il 30% temporaneo a metà (15% Val, 15% Test)
val_set, test_set = train_test_split(
    temp_set, 
    test_size=0.50, 
    random_state=42, 
    stratify=temp_set['label']
)


print(f"Split completato:")
print(f"- Train: {len(train_set)} righe")
print(f"- Val:   {len(val_set)} righe")
print(f"- Test:  {len(test_set)} righe\n")

Split completato:
- Train: 8703 righe
- Val:   1865 righe
- Test:  1866 righe



In [19]:
# Salvataggio su file CSV
train_set.to_csv("../../data/ground_truth/GT_train/train.csv", index=False)
val_set.to_csv("../../data/ground_truth/GT_train/val.csv", index=False)
test_set.to_csv("../../data/ground_truth/GT_train/test.csv", index=False)